In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
year = "1960-61"

In [ ]:
webpage = f"https://en.wikipedia.org/wiki/{year}_La_Liga"

In [ ]:
content = requests.get(webpage).text

In [ ]:
soup = BeautifulSoup(content, "html.parser")

In [ ]:
tables = soup.find_all("table")

In [ ]:
for table in tables:
    if "Home \\ Away" in table.text:
        results_table = table

results_table

In [ ]:
results_table_rows = results_table.findChild().find_all("tr")
results_table_rows

In [ ]:
away_teams = []
home_teams = []

Getting Away Team Names

In [ ]:
away_teams_row = results_table_rows[0].find_all("th")
away_teams_row

In [ ]:
for i in range(1, len(away_teams_row)):
    team_name = away_teams_row[i].findChild()["title"]
    away_teams.append(team_name)

In [ ]:
away_teams

Getting Home Teams

In [ ]:
for i in range(1, len(results_table_rows)):
    team_name = results_table_rows[i].find("th").findChild()["title"]
    home_teams.append(team_name)

In [ ]:
home_teams

Getting Match Results

In [ ]:
teams_matches = {}

In [ ]:
for i in range(1, len(results_table_rows)):
    tds = results_table_rows[i].find_all("td")
    for j in range(len(tds)):
        if "—" in tds[j].text.strip():
            continue
        try:
            score = tds[j].find("a").text.strip()
            original = teams_matches.get(home_teams[i - 1], [])
            original.append([away_teams[j], score.strip()])
            teams_matches[home_teams[i - 1]] = original
        except:
            score = tds[j].text.strip()
            original = teams_matches.get(home_teams[i - 1], [])
            original.append([away_teams[j], score.strip()])
            teams_matches[home_teams[i - 1]] = original

In [ ]:
df_dict = {"Home": [], "Away": [], "Score": []}

In [ ]:
for key, values in teams_matches.items():
    for value in values:
        df_dict["Home"].append(key)
        df_dict["Away"].append(value[0])
        df_dict["Score"].append(value[1])

In [ ]:
df = pd.DataFrame(df_dict)
df.to_csv(f"./results/results_{year}.csv", index=False)